# [Tor Node Lookup](https://www.torproject.org/)

Tor aims to conceal its users' identities and their online activity from surveillance and traffic analysis by separating identification and routing. It is an implementation of onion routing, which encrypts and then randomly bounces communications through a network of relays run by volunteers around the globe.


This is a continuation of PyConAfrica Maltego OSINT Workshop. 


![Tor](https://i.imgur.com/J1K4uW5.png)

# How can we identify if an IP Address is a Tor Node ? 

Tor mantains a list of current Tor Exit Nodes on its website. We can see then on this URL. https://check.torproject.org/exit-addresses The list is output as follows. 

# OSINT is usually about finding Data readily avalable online

Mostly Dealing with APIs or HTTP Calls , so read up on 
* Requests
* Rest Client in VSCode 


## So how can we create a Transform that will Automatically check if an ip address is a Tor Node? 

First you need to figure out exactly how you want to retrieve data

In [7]:
import requests
def downloadTorExitList():
        try:
            httpResponse = requests.get("https://check.torproject.org/exit-addresses")
            print(httpResponse.text)
        except Exception as e:
            print(e)

In [8]:
downloadTorExitList()

ExitNode 0011BD2485AD45D984EC4159C88FC066E5E3300E
Published 2019-08-09 20:19:30
LastStatus 2019-08-09 21:03:27
ExitAddress 162.247.74.201 2019-08-09 21:03:47
ExitNode 0020D8A204A81FC8C8A1D79E2324621705DE4F2F
Published 2019-08-09 17:08:07
LastStatus 2019-08-09 18:03:30
ExitAddress 178.17.170.116 2019-08-09 18:06:58
ExitNode 003D78825E0B9609EECFF5E4E0529717772E53C7
Published 2019-08-09 14:45:35
LastStatus 2019-08-09 15:02:27
ExitAddress 104.218.63.73 2019-08-09 15:09:20
ExitNode 0077BCBA7244DB3E6A5ED2746E86170066684887
Published 2019-08-09 09:38:00
LastStatus 2019-08-09 10:02:26
ExitAddress 199.249.230.103 2019-08-09 10:02:38
ExitNode 00CCE6A84E6D63A1A42E105839BC8ED5D4B16669
Published 2019-08-09 17:15:17
LastStatus 2019-08-09 18:03:30
ExitAddress 89.236.112.99 2019-08-09 18:11:30
ExitNode 0111BA9B604669E636FFD5B503F382A4B7AD6E80
Published 2019-08-09 12:37:22
LastStatus 2019-08-09 13:02:26
ExitAddress 176.10.104.240 2019-08-09 13:09:28
ExitNode 01181B31BE5860C7D66DA88F88AD522C06470FD9
Pub

Let's write our output to file and search from our file

In [9]:
import requests
def downloadUpdatedList():
        try:
            httpResponse = requests.get("https://check.torproject.org/exit-addresses")
            outputFile = open("tor_exit_nodes.csv",'w')
            outputFile.write(httpResponse.text)
            outputFile.close()
            httpResponse.close()
        except Exception as e:
            print(e)

## How can we check if an ip address is in our file

In [10]:
downloadUpdatedList()
ip_address = "199.249.230.103"
with open('tor_exit_nodes.csv', 'r') as fp:
    for row in fp:
        if ip_address in row:
            print("found you")

found you


# Now for the Transform

Add a new transform by creating a new python file in the "transforms" folder of your directory.

In order for a new transform to be discovered, we must create a new file that:

    Is inside the "transforms" folder
    Contains a class with the same name as the filename
    The class inherits from the DiscoverableTransform class

Create file

*pyconafrica/transforms/LookUpTorExit.py*

add code below



In [11]:
from maltego_trx.entities import Phrase
from maltego_trx.transform import DiscoverableTransform
from maltego_trx.maltego import UIM_PARTIAL
import requests

class LookUpTorExit(DiscoverableTransform):
    """
    Check if an IP is a Tor Exit Node and Returns a Phrase
    """

    @classmethod
    def create_entities(cls, request, response):
        ip_address = request.Value
        
        # cls.downloadUpdatedList()
        
        try:
            with open('tor_exit_nodes.csv', 'r') as fp:
                for row in fp:
                    if ip_address in row:
                        tor_node = response.addEntity(Phrase, 'Tor Exit Node')
                        tor_node.setNote("This is a Note")
            response.addEntity(Phrase, "Not a Tor")
        except Exception as e:    
            response.addUIMessage("An error occurred ", messageType=UIM_PARTIAL)
            

    def downloadUpdatedList():
        try:
            httpResponse = requests.get("https://check.torproject.org/exit-addresses")
            outputFile = open("tor_exit_nodes.csv",'w')
            outputFile.write(httpResponse.text)
            outputFile.close()
            httpResponse.close()
        except Exception as e:
            print(e)

* How can you make It Efficient? 
* Do we have to Download the file every Time? 
